## Deliverable 2. Create a Customer Travel Destinations Map.

In [35]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [36]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,mataura,NZ,-46.1927,168.8643,58.48,53,67,7.92,broken clouds
1,1,punta arenas,CL,-53.1500,-70.9167,53.71,58,0,23.02,clear sky
2,2,cabo san lucas,MX,22.8909,-109.9124,74.52,66,0,1.99,clear sky
3,3,clyde river,CA,70.4692,-68.5914,-0.17,78,97,16.11,overcast clouds
4,4,almaznyy,RU,48.0448,40.0450,30.34,98,100,7.14,snow


In [37]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation?"))
max_temp = float(input("What is the maximum temperature you would like for your vacation?"))

What is the minimum temperature you would like for your vacation?60
What is the maximum temperature you would like for your vacation?100


In [38]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,cabo san lucas,MX,22.8909,-109.9124,74.52,66,0,1.99,clear sky
5,5,hermanus,ZA,-34.4187,19.2345,72.90,86,100,11.81,overcast clouds
6,6,faanui,PF,-16.4833,-151.7500,81.61,69,90,10.89,overcast clouds
8,8,hobart,AU,-42.8794,147.3294,61.05,66,40,3.00,light rain
10,10,avarua,CK,-21.2078,-159.7750,75.25,94,20,2.30,few clouds
...,...,...,...,...,...,...,...,...,...,...
713,713,wattegama,LK,6.7989,81.4808,85.12,47,7,9.08,clear sky
715,715,mankono,CI,8.0586,-6.1897,70.30,41,4,3.11,clear sky
717,717,inverell,AU,-29.7833,151.1167,88.66,40,100,5.99,overcast clouds
720,720,luba,GQ,3.4568,8.5547,81.32,81,92,3.58,overcast clouds


In [39]:
# 4a. Determine if there are any empty rows.
clean_df.isnull()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False
10,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
713,False,False,False,False,False,False,False,False,False,False
715,False,False,False,False,False,False,False,False,False,False
717,False,False,False,False,False,False,False,False,False,False
720,False,False,False,False,False,False,False,False,False,False


In [40]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()
clean_df.count()

City_ID                358
City                   358
Country                358
Lat                    358
Lng                    358
Max Temp               358
Humidity               358
Cloudiness             358
Wind Speed             358
Current Description    358
dtype: int64

In [41]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,cabo san lucas,MX,74.52,clear sky,22.8909,-109.9124,
5,hermanus,ZA,72.90,overcast clouds,-34.4187,19.2345,
6,faanui,PF,81.61,overcast clouds,-16.4833,-151.7500,
8,hobart,AU,61.05,light rain,-42.8794,147.3294,
10,avarua,CK,75.25,few clouds,-21.2078,-159.7750,
11,georgetown,MY,89.53,few clouds,5.4112,100.3354,
12,vanimo,PG,82.56,light rain,-2.6741,141.3028,
13,ahipara,NZ,70.05,overcast clouds,-35.1667,173.1667,
15,saldanha,ZA,64.45,clear sky,-33.0117,17.9442,
18,guarumal,PA,79.93,overcast clouds,8.3500,-82.5333,


In [42]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel Not Found... Skipping...")
        

Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...


In [43]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,cabo san lucas,MX,74.52,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
5,hermanus,ZA,72.90,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
6,faanui,PF,81.61,overcast clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
8,hobart,AU,61.05,light rain,-42.8794,147.3294,Mantra on Collins Hobart
10,avarua,CK,75.25,few clouds,-21.2078,-159.7750,Paradise Inn
...,...,...,...,...,...,...,...
705,keti bandar,PK,76.93,overcast clouds,24.1447,67.4497,Haji laloo memon house
706,amahai,ID,88.02,overcast clouds,-3.3333,128.9167,HOTEL ISABELA
709,moultrie,US,60.39,clear sky,31.1799,-83.7891,Moultrie Inn
715,mankono,CI,70.30,clear sky,8.0586,-6.1897,Hôtel cotonputeur


In [44]:
# 8a. Create the output File (CSV)
output_data_file = "./data/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [45]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name} at {Max Temp}</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [46]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure()
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))